In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import os

from scipy.optimize import curve_fit

from functionsFIMS import getAnalysisNumbers, plotPolyExamples
from runDataClass import runData

In [ ]:
if __name__ == '__main__':
    plotPolyExamples()

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(runNos[-1])
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
def setScale(independent, dependent, xScale, yScale):
    """
    args: 
    A list containing the values of the independent variable (list or dataframe).
    A list containing the values of the dependent variable (list or dataframe).
    The scale for tick marks on the x-axis (int).
    The scale for tick marks on the y-axis (int).
    
    Returns: a dictionary with a list of data values for the x-axis, a list of data values for 
    the y-axis, the startinng value for x, and the starting value for y. 
    
    All four dictionary items intended for use in a matplot.lib xticks() or yticks() command.
    """
    try:
        xMin = independent.min(axis = 0)
        xMax = independent.max(axis = 0)
        yMin = dependent.min(axis = 0)
        yMax = dependent.max(axis = 0)
    
    except:
        xMin = min(independent)
        xMax = max(independent)
        yMin = min(dependent)
        yMax = max(dependent)
    
    xPlotMin = xScale*round(xMin/xScale, 0)
    xPlotMax = xMax+ xScale
    yPlotMin = yScale*round(yMin/yScale, 0)
    yPlotMax = yMax + yScale
    
    xTicks = np.arange(xPlotMin, xPlotMax, xScale)
    yTicks = np.arange(yPlotMin, yPlotMax, yScale)
    
    scale = {
    'xPlot': xTicks,
    'yPlot': yTicks,
    'xMin': xPlotMin,
    'yMin': yPlotMin,
    }

    return scale

In [ ]:
def multiPlotSimRange(dataSet, independent, dependent, save = False):
    """
    args:
    Four total arguments: three lists and a boolean.
    
    1) A list containting a number of lists equal to the the number of data sets the user
    wishes to plot. Each list should contain the following:
        initial data set (int)
        final data set (int)
        label for the legend (string)
        color for the data (string)

    2) A list of the details for the independent axis:
        parameter name (should match the name in the metaData) (string)
        axis scale (int or float)
        axis units (string)

    3) A list of the details for the dependent axis:
        parameter name (should match the name in the metaData) (string)
        axis scale (int or float)
        axis units (string)

    4) Indicator of whether the user wants to save the plot (boolean)

    
    Saves the plot under a directory labeled Plots with a name of the form y_vs_x.png.
    
    The initial and final data sets used for the first and second position of the lists
    within dataSet should form a continuous, discreet set (IE: 5, 8 should look like 
    5, 6, 7, 8, not 5, 7, 8)
    
    micro symbol unicode is: \u03bc
    
    """

    try:
        if type(dataSet[0]) == type(dataSet[1]) == list:
            print('Acquiring Data')
    except: 
        print('Error: dataSet wrong data type (should be list of lists)')
        return -1
    
    if not os.path.exists('./Plots'):
            os.mkdir('./Plots')

    setNum = 0
    xExtremes = []
    yExtremes = []
    
    while setNum < len(dataSet):
        indepData = createDataList(dataSet[setNum][0], dataSet[setNum][1], independent[0])
        depData = createDataList(dataSet[setNum][0], dataSet[setNum][1], dependent[0]) 

        xExtremes.extend([min(indepData), max(indepData)])
        yExtremes.extend([min(depData), max(depData)])
        
        plt.scatter(indepData, depData, marker = 'x', s = 8, label = dataSet[setNum][2], color = dataSet[setNum][3])

        setNum += 1
        print(f'Data set {setNum} read and plotted succesfully')
    
    setNum -= 1
    
    Scales = setScale(xExtremes, yExtremes, independent[1], dependent[1])
    plt.title(f'{dependent[0]} vs {independent[0]}')
    plt.legend(loc = 'upper right', prop = {'size': 8})
    plt.xlabel(f'{independent[0]} {independent[2]}')
    plt.xticks(Scales['xPlot'])
    plt.ylabel(f'{dependent[0]} {dependent[2]}')
    plt.yticks(Scales['yPlot'])

    if save:
        plt.savefig(f'./Plots/{dependent[0]}_vs_{independent[0]}.png')
    plt.show()

In [ ]:
#Example use of multiplotSimRange()
data = [
    [1, 10, 'Grid Standoff = 100 (\u03bcm)', 'blue'],
    [11, 20, 'Grid Standoff = 100 (\u03bcm)', 'purple'],
    [21, 30, 'Grid Standoff = 100 (\u03bcm)', 'red']
    ]
independent = ['Hole Radius', 5, '(\u03bcm)']
dependent = ['Field Bundle Radius', 2, '(\u03bcm)']

multiPlotSimRange(data, independent, dependent, False)

In [ ]:
if __name__ == '__main__':
    #runNos = getAnalysisNumbers()
    simData = runData()
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)
    
    simData.plotCellGeometry()

In [ ]:
simData.plot2DFieldLines('Cathode')